<a href="https://colab.research.google.com/github/subeans/security_project/blob/main/isolation_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/security_project/04_hashed.csv")


In [ ]:
dataset_split = dataset.iloc[:1000000]

In [ ]:
datset_split_without_Rdate = dataset_split.drop('Rdate',axis=1)

In [ ]:
datset_split_without_Rdate

,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,154.58.159.102,103.177.12.42,6,52897,445,2,None,US
1,154.58.159.20,125.66.92.196,6,60579,445,2,None,DE
2,154.58.159.164,117.121.178.223,6,63831,445,2,None,US
3,154.58.159.165,205.34.95.97,6,55241,445,2,None,US
4,154.58.159.102,93.56.164.131,6,52898,445,2,None,US
...,...,...,...,...,...,...,...,...
999995,154.58.159.164,217.55.91.177,6,64757,445,2,None,US
999996,154.58.159.102,88.81.214.201,6,57235,445,2,None,US
999997,154.58.159.20,254.133.112.214,6,64907,445,2,None,US
999998,154.58.159.164,252.184.8.211,6,64759,445,2,None,SE


In [ ]:
dataset_split

,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,2.021041e+13,154.58.159.102,103.177.12.42,6,52897,445,2,None,US
1,2.021041e+13,154.58.159.20,125.66.92.196,6,60579,445,2,None,DE
2,2.021041e+13,154.58.159.164,117.121.178.223,6,63831,445,2,None,US
3,2.021041e+13,154.58.159.165,205.34.95.97,6,55241,445,2,None,US
4,2.021041e+13,154.58.159.102,93.56.164.131,6,52898,445,2,None,US
...,...,...,...,...,...,...,...,...,...
999995,2.021041e+13,154.58.159.164,217.55.91.177,6,64757,445,2,None,US
999996,2.021041e+13,154.58.159.102,88.81.214.201,6,57235,445,2,None,US
999997,2.021041e+13,154.58.159.20,254.133.112.214,6,64907,445,2,None,US
999998,2.021041e+13,154.58.159.164,252.184.8.211,6,64759,445,2,None,SE


In [ ]:
# dataset_train = dataset.iloc[:100]
# dataset_test = dataset.iloc[100:150]

In [ ]:
""" 
IPv4인 경우 각 그룹을 균등화하고 IPv6 길이와 일치하도록 0을 남겨둡니다. 
IPv6인 경우 모두 소문자로 변환합니다. 
""" 
def transform_ip(ip): 
  IPV6_LENGTH = 39
  IPV4_GROUP_LENGTH = 3 # IPv4의 각 그룹은 이 길이입니다.
  if len(ip) < IPV6_LENGTH: 
      # IPv4 주소 
      groups = ip.split( "." ) 
      equalize_group_length = "".join( map( lambda group: group.zfill(3), groups )) 
      return equalize_group_length 
  else: 
      return list(ip.lower())

In [ ]:
"""
Converts the ipAddress column of pandas DataFrame df, to one-hot
Also returns the encoder used
"""
# from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import OneHotEncoder

def one_hot_ip(df):
  # enc = CategoricalEncoder()
  enc = OneHotEncoder()
  df['src_ip'] = df.src_ip.apply(lambda ip : transform_ip(ip))
  # print(df['src_ip'])
  df['dst_ip'] = df.dst_ip.apply(lambda ip : transform_ip(ip)).apply(pd.Series)
  # print(df['dst_ip'])
  # X_ip = enc.fit_transform( df ).toarray()
  X_ip=[]
  return X_ip, enc ,df

In [ ]:
X_ip,enc,dataset_ip = one_hot_ip(dataset_split)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
import numpy as np
arr = np.array(dataset_ip)
dataset_ip = pd.DataFrame(arr,columns=dataset_ip.columns)


,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,2.02104e+13,154058159102,103177012042,6,52897,445,2,None,US
1,2.02104e+13,154058159020,125066092196,6,60579,445,2,None,DE
2,2.02104e+13,154058159164,117121178223,6,63831,445,2,None,US
3,2.02104e+13,154058159165,205034095097,6,55241,445,2,None,US
4,2.02104e+13,154058159102,093056164131,6,52898,445,2,None,US
...,...,...,...,...,...,...,...,...,...
999995,2.02104e+13,154058159164,217055091177,6,64757,445,2,None,US
999996,2.02104e+13,154058159102,088081214201,6,57235,445,2,None,US
999997,2.02104e+13,154058159020,254133112214,6,64907,445,2,None,US
999998,2.02104e+13,154058159164,252184008211,6,64759,445,2,None,SE


In [ ]:
from sklearn.preprocessing import LabelEncoder
job_encode = LabelEncoder()
dataset_ip['src_country'] = job_encode.fit_transform(dataset_ip['src_country'])
dataset_ip['dst_country'] = job_encode.fit_transform(dataset_ip['dst_country'])
dataset_ip.tail()

,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
999995,2.02104e+13,154058159164,217055091177,6,64757,445,2,92,204
999996,2.02104e+13,154058159102,088081214201,6,57235,445,2,92,204
999997,2.02104e+13,154058159020,254133112214,6,64907,445,2,92,204
999998,2.02104e+13,154058159164,252184008211,6,64759,445,2,92,173
999999,2.02104e+13,072192214061,020250066003,6,54199,135,2,67,204


,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,2.02104e+13,154058159102,103177012042,6,52897,445,2,92,204
1,2.02104e+13,154058159020,125066092196,6,60579,445,2,92,49
2,2.02104e+13,154058159164,117121178223,6,63831,445,2,92,204
3,2.02104e+13,154058159165,205034095097,6,55241,445,2,92,204
4,2.02104e+13,154058159102,093056164131,6,52898,445,2,92,204
...,...,...,...,...,...,...,...,...,...
999995,2.02104e+13,154058159164,217055091177,6,64757,445,2,92,204
999996,2.02104e+13,154058159102,088081214201,6,57235,445,2,92,204
999997,2.02104e+13,154058159020,254133112214,6,64907,445,2,92,204
999998,2.02104e+13,154058159164,252184008211,6,64759,445,2,92,173


In [ ]:
from sklearn.ensemble import IsolationForest

model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1,
                      max_features=9, contamination='auto')

model.fit(dataset_ip.to_numpy())


IsolationForest(max_features=9, n_jobs=-1)

In [ ]:
score = model.decision_function(dataset_ip.to_numpy())
anomaly = model.predict(dataset_ip.to_numpy())


In [ ]:
dataset_ip['scores']= score
dataset_ip['anomaly']= anomaly

anomaly_data = dataset_ip.loc[dataset_ip['anomaly']==-1] # 이상값은 -1으로 나타낸다.
anomaly_data

,Rdate,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country,scores,anomaly
108,2.02104e+13,173224148008,045111016083,17,53,42540,2,122,105,-0.017244,-1
213,2.02104e+13,010197253034,045111016244,6,50004,63189,2,11,105,-0.038782,-1
233,2.02104e+13,245239011096,072192214231,6,3618,44223,2,67,105,-0.004423,-1
532,2.02104e+13,045111016058,077061151025,6,50306,55002,1,67,204,-0.014746,-1
806,2.02104e+13,045111016060,077061151025,6,3561,55002,1,67,204,-0.043681,-1
...,...,...,...,...,...,...,...,...,...,...,...
999933,2.02104e+13,173224148008,072192214061,17,53,45626,2,122,105,-0.053195,-1
999947,2.02104e+13,121007158006,045111016021,6,43210,57417,2,106,105,-0.009762,-1
999975,2.02104e+13,253142148158,045111016071,6,2048,1433,2,22,105,-0.021834,-1
999982,2.02104e+13,248105148139,045111016164,6,52437,50022,2,122,105,-0.016509,-1
